In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import time
import random
from IPython.display import clear_output

class GridEnvironment:
    def __init__(self, grid_size, start, goal, obstacles, coins=None, warps=None, jumps=None):
        """
        Initialize the grid environment.
        Args:
            grid_size (tuple): Size of the grid (rows, columns).
            start (tuple): Starting position of the agent.
            goal (tuple): Goal position.
            obstacles (list of tuples): List of obstacle positions.
            coins (list of tuples): List of coin positions.
            warps (list of tuples): Warp pairs as [(source1, dest1), (source2, dest2)].
            jumps (list of tuples): List of jump positions.
        """
        self.grid_size = grid_size
        self.start = start
        self.goal = goal
        self.obstacles = obstacles
        self.coins = coins if coins else []
        self.warps = warps if warps else []
        self.jumps = jumps if jumps else []
        self.reset()

    def reset(self):
        """Reset the environment to the initial state."""
        self.agent_pos = self.start
        self.remaining_coins = set(self.coins)  # Reset coins
        return self.agent_pos
    
    
    
    def step(self, action): 

        row,col = self.agent_pos

        number_of_actions = 1

        # Mngelompat
        if self.agent_pos in self.jumps:
            number_of_actions = 2

        if action == 0: # up
            row = max(0, row - number_of_actions)
        elif action == 1: # right
            col = min(self.grid_size[1] - 1, col + number_of_actions)
        elif action == 2: # down
            row = min(self.grid_size[0] - 1, row + number_of_actions)
        elif action == 3: # left
            col = max(0, col - number_of_actions)
        else:
            raise ValueError(f"Invalid action: {action}")

        next_pos = (row, col)

        # Check if the agent steps on a warp
        for w1, w2 in self.warps:
            if next_pos == w1:
                next_pos = w2
                break
            elif next_pos == w2:
                next_pos = w1
                break   

        # Check if the next position is valid
        if next_pos in self.obstacles :
            next_pos = self.agent_pos
        
        self.agent_pos = next_pos

        # Calculate reward
        reward = 0
        if next_pos == self.goal:
            if len(self.remaining_coins) == 0:
                reward += 1  # Reward for reaching the goal with all coins collected
                done = True
            else:
                reward += -1  # Penalty for reaching the goal without collecting all coins
                done = False
        else:
            reward += -0.1  # Small penalty for each step
            done = False
        
        # Check if the agent collects a coin
        if next_pos in self.remaining_coins:
            self.remaining_coins.remove(next_pos)
            reward += 0.5  # Bonus reward for collecting a coin

        return next_pos, reward, done


    def render(self):
        """Render the grid."""
        # Initialize the grid with longer string support
        grid = np.full(self.grid_size, ". ", dtype="<U2")  # Allow strings up to 2 characters

        grid[self.goal] = "G "
        for obs in self.obstacles:
            grid[obs] = "X "
        for coin in self.remaining_coins:
            grid[coin] = "C "
        for i, (w1, w2) in enumerate(self.warps, start=1):
            grid[w1] = f"W{i}"
            grid[w2] = f"W{i}"
        for jump in self.jumps:
            grid[jump] = "J "
        grid[self.agent_pos] = "A "
        print("\n".join([" ".join(row) for row in grid]))
        print()

grid_size = (10, 10)
start = (0, 0)
goal = (9, 9)
obstacles = [
    (1, 3), (1, 4), (1, 5), (2, 3), (3, 1), (3, 2), (4, 6), (6, 7), (7, 8)
]
coins = [(0, 9), (3, 3), (7, 0), (9, 5)]
warps = [
    ((2, 9), (8, 8)),
    ((5, 5), (1, 1))
]
jumps = [(2, 7), (4, 4), (7, 6), (9, 2)]

env = GridEnvironment(grid_size, start, goal, obstacles, coins, warps, jumps)

# lanjutkan kode di atas, tapi posisi-posisi special tiles (agent, goal, coins, etc.) dan ukuran grid jangan di ganti



class SARSAAgent:
    def __init__(self, grid_size, actions, alpha=0.1, gamma=0.9, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.99):
        """
        Initialize the SARSA agent.
        Args:
            grid_size (tuple): Size of the grid.
            actions (int): Number of possible actions.
            alpha (float): Learning rate.
            gamma (float): Discount factor.
            epsilon (float): Initial exploration rate.
            epsilon_min (float): Minimum exploration rate.
            epsilon_decay (float): Decay factor for exploration rate.
        """
        self.q_table = np.zeros((*grid_size, actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.actions = actions

    def choose_action(self, state):
        """Choose an action using epsilon-greedy policy."""
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.actions)
        return np.argmax(self.q_table[state])

    def update(self, state, action, reward, next_state, next_action):
        """Update the Q-value using the SARSA update rule."""
        current_q = self.q_table[state][action]
        next_q = self.q_table[next_state][next_action]
        self.q_table[state][action] += self.alpha * (reward + self.gamma * next_q - current_q)

def train(env, agent, episodes=500):
    """
    Train the agent in the environment using SARSA with epsilon decay.
    Args:
        env (GridEnvironment): The environment.
        agent (SARSAAgent): The SARSA agent.
        episodes (int): Number of training episodes.
    """
    for episode in range(episodes):
        state = env.reset()
        action = agent.choose_action(state)
        done = False

        while not done:
            next_state, reward, done = env.step(action)
            next_action = agent.choose_action(next_state)
            agent.update(state, action, reward, next_state, next_action)
            state, action = next_state, next_action

        # Decay epsilon
        agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)

def simulate(env, agent, delay=0.5):
    """
    Simulate the agent navigating the grid.
    Args:
        env (GridEnvironment): The environment.
        agent (SARSAAgent): The trained SARSA agent.
        delay (float): Delay between steps (in seconds).
    """
    state = env.reset()
    done = False

    step = 0

    while not done:
        clear_output(wait=True)
        env.render()
        if agent.q_table[state].sum() == 0:
            action = random.choice([0, 1, 2, 3])
        else:
            action = np.argmax(agent.q_table[state])
        state, _, done = env.step(action)
        time.sleep(delay)
        step += 1

    clear_output(wait=True)
    env.render()
    print("Goal reached!")
    print(f"Step : {step}")



env.render()

A  .  .  .  .  .  .  .  .  C 
.  W2 .  X  X  X  .  .  .  . 
.  .  .  X  .  .  .  J  .  W1
.  X  X  C  .  .  .  .  .  . 
.  .  .  .  J  .  X  .  .  . 
.  .  .  .  .  W2 .  .  .  . 
.  .  .  .  .  .  .  X  .  . 
C  .  .  .  .  .  J  .  X  . 
.  .  .  .  .  .  .  .  W1 . 
.  .  J  .  .  C  .  .  .  G 



In [ ]:
grid_size = (10, 10)
start = (0, 0)
goal = (9, 9)
obstacles = [
    (1, 3), (1, 4), (1, 5), (2, 3), (3, 1), (3, 2), (4, 6), (6, 7), (7, 8)
]
coins = [(0, 9), (3, 3), (7, 0), (9, 5)]
warps = [
    ((2, 9), (8, 8)),
    ((5, 5), (1, 1))
]
jumps = [(2, 7), (4, 4), (7, 6), (9, 2)]

env = GridEnvironment(grid_size, start, goal, obstacles, coins, warps, jumps)
agent = SARSAAgent(grid_size, actions=4)

train(env, agent, episodes=1000)

In [53]:
# Simulate agent (post-RL)
simulate(env, agent)

.  A  .  .  .  .  .  .  .  C 
.  W2 .  X  X  X  .  .  .  . 
.  .  .  X  .  .  .  J  .  W1
.  X  X  C  .  .  .  .  .  . 
.  .  .  .  J  .  X  .  .  . 
.  .  .  .  .  W2 .  .  .  . 
.  .  .  .  .  .  .  X  .  . 
C  .  .  .  .  .  J  .  X  . 
.  .  .  .  .  .  .  .  W1 . 
.  .  J  .  .  C  .  .  .  G 



KeyboardInterrupt: 

In [54]:
# Simulate agent (pre-RL)
agent2 = SARSAAgent(grid_size, actions=4)
simulate(env, agent2)

.  .  .  .  .  .  .  .  .  C 
.  W2 .  X  X  X  .  .  .  . 
.  .  .  X  .  .  .  J  .  W1
.  X  X  C  .  .  .  .  .  . 
.  .  .  .  J  .  X  .  .  . 
.  .  .  .  .  W2 .  .  .  . 
.  .  .  .  .  .  .  X  .  . 
C  A  .  .  .  .  J  .  X  . 
.  .  .  .  .  .  .  .  W1 . 
.  .  J  .  .  C  .  .  .  G 



KeyboardInterrupt: 